In [1]:
# 1. Imports & Settings
import json
import os
from pathlib import Path
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
)
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering

warnings.filterwarnings("ignore")
RANDOM_STATE = 42

# Paths
BASE_DIR = Path(".")
DATA_DIR = BASE_DIR / "data"
ARTIFACTS_DIR = BASE_DIR / "artifacts"
FIGS_DIR = ARTIFACTS_DIR / "figures"
LABELS_DIR = ARTIFACTS_DIR / "labels"
for p in (ARTIFACTS_DIR, FIGS_DIR, LABELS_DIR):
    p.mkdir(parents=True, exist_ok=True)

DEFAULT_DATA_FILES = [
    DATA_DIR / "S07-hw-dataset-02.csv",
    DATA_DIR / "S07-hw-dataset-03.csv",
    DATA_DIR / "S07-hw-dataset-04.csv",
]

DATA_FILES = DEFAULT_DATA_FILES

# Helper printing
def info(msg: str):
    print(f"[INFO] {msg}")

In [2]:
# 2. Utility functions
def safe_read_csv(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"Dataset file not found: {path}")
    return pd.read_csv(path)

def build_preprocessor(df: pd.DataFrame):
    """
    Build ColumnTransformer:
    - Numeric: SimpleImputer(mean) -> StandardScaler
    - Categorical: SimpleImputer(constant) -> OneHotEncoder(handle_unknown='ignore')
    Returns transformer and list of numeric/categorical column names.
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    # exclude sample_id if present
    numeric_cols = [c for c in numeric_cols if c != "sample_id"]
    cat_cols = [c for c in df.columns if c not in numeric_cols and c != "sample_id"]
    # numeric pipeline
    numeric_transformer = (
        ("num", SimpleImputer(strategy="mean"), numeric_cols)
        if numeric_cols
        else None
    )
    # categorical pipeline
    categorical_transformer = (
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)
        if cat_cols
        else None
    )
    transformers = []
    if numeric_transformer:
        transformers.append(numeric_transformer)
    if categorical_transformer:
        transformers.append(categorical_transformer)
    # Build ColumnTransformer that imputes and encodes; scaling applied after transform
    from sklearn.compose import ColumnTransformer

    ct = ColumnTransformer(transformers, remainder="drop", sparse_threshold=0)
    return ct, numeric_cols, cat_cols

def preprocess_fit_transform(ct: ColumnTransformer, df: pd.DataFrame):
    """
    Fit ColumnTransformer on df and return transformed numpy array and feature names.
    Then apply StandardScaler to numeric+encoded features.
    """
    X_pre = ct.fit_transform(df)
    # Build feature names for transformed output (best-effort)
    feat_names = []
    for name, transformer, cols in ct.transformers_:
        if transformer is None:
            continue
        if hasattr(transformer, "get_feature_names_out"):
            try:
                out_names = transformer.get_feature_names_out(cols)
            except Exception:
                out_names = cols
        else:
            out_names = cols
        feat_names.extend([str(n) for n in out_names])
    # scale
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_pre)
    return X_scaled, feat_names, ct, scaler

def preprocess_transform(ct: ColumnTransformer, scaler: StandardScaler, df: pd.DataFrame):
    X_pre = ct.transform(df)
    X_scaled = scaler.transform(X_pre)
    return X_scaled

def compute_internal_metrics(X: np.ndarray, labels: np.ndarray, consider_noise_as_cluster=False):
    """
    Compute silhouette, davies_bouldin, calinski_harabasz.
    For labels where one cluster or all noise, return None appropriately.
    """
    metrics = {"silhouette": None, "davies_bouldin": None, "calinski_harabasz": None}
    # number of valid clusters
    unique_labels = np.unique(labels)
    # If DBSCAN noise present and consider_noise_as_cluster False, we compute metrics on non-noise only
    if -1 in unique_labels and (not consider_noise_as_cluster):
        non_noise_mask = labels != -1
        if non_noise_mask.sum() < 2:
            return metrics, {"n_points_non_noise": int(non_noise_mask.sum()), "n_clusters_non_noise": 0}
        X_eval = X[non_noise_mask]
        labels_eval = labels[non_noise_mask]
    else:
        X_eval = X
        labels_eval = labels
    n_clusters = len(set(labels_eval))
    if n_clusters <= 1 or X_eval.shape[0] <= 1:
        return metrics, {"n_points_non_noise": int((labels != -1).sum()), "n_clusters_non_noise": n_clusters}
    try:
        metrics["silhouette"] = float(silhouette_score(X_eval, labels_eval))
    except Exception:
        metrics["silhouette"] = None
    try:
        metrics["davies_bouldin"] = float(davies_bouldin_score(X_eval, labels_eval))
    except Exception:
        metrics["davies_bouldin"] = None
    try:
        metrics["calinski_harabasz"] = float(calinski_harabasz_score(X_eval, labels_eval))
    except Exception:
        metrics["calinski_harabasz"] = None
    return metrics, {"n_points_non_noise": int((labels != -1).sum()), "n_clusters_non_noise": n_clusters}

def pca_2d(X: np.ndarray, random_state=RANDOM_STATE):
    pca = PCA(n_components=2, random_state=random_state)
    X2 = pca.fit_transform(X)
    return X2, pca

def save_fig(fig, path: Path):
    fig.savefig(path, dpi=150, bbox_inches="tight")
    plt.close(fig)



In [3]:
# 3. Main experiment per dataset
metrics_summary = {}
best_configs = {}

processed_datasets = []  # store dataset names processed

for idx, data_file in enumerate(DATA_FILES, start=1):
    dataset_name = data_file.stem
    info(f"Processing dataset {idx}: {data_file}")
    if not data_file.exists():
        info(f"File {data_file} not found — skipping")
        continue
    df = safe_read_csv(data_file)
    processed_datasets.append(dataset_name)

    info("Basic EDA:")
    info(f" - shape: {df.shape}")
    info(f" - head:\n{df.head().to_string(index=False)}")
    info(f" - dtypes:\n{df.dtypes}")
    # missing
    miss = df.isna().sum()
    info(f" - missing summary (top):\n{miss[miss>0].sort_values(ascending=False).head().to_string() or 'No missing values'}")

    # Separate sample_id if exists
    sample_ids = df["sample_id"] if "sample_id" in df.columns else pd.Series(np.arange(len(df)), name="sample_id")
    X_df = df.drop(columns=["sample_id"]) if "sample_id" in df.columns else df.copy()

    info("Building preprocessor (imputation, encoding)...")
    ct, numeric_cols, cat_cols = build_preprocessor(X_df)
    try:
        X_scaled, feat_names, ct_fitted, scaler_fitted = preprocess_fit_transform(ct, X_df)
        info(f" - transformed shape: {X_scaled.shape}")
    except Exception as exc:
        info(f"Preprocessing failed: {exc}")
        continue

    info("Searching K for KMeans (2..12)")
    k_range = list(range(2, min(13, max(3, X_scaled.shape[0]//5))))  # adaptive upper bound
    k_range = k_range if k_range else [2,3,4]
    k_metrics = []
    for k in k_range:
        try:
            km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
            labels_k = km.fit_predict(X_scaled)
            met, extra = compute_internal_metrics(X_scaled, labels_k, consider_noise_as_cluster=True)
            k_metrics.append({"k": k, **met, "n_clusters": len(set(labels_k))})
        except Exception as exc:
            k_metrics.append({"k": k, "silhouette": None, "davies_bouldin": None, "calinski_harabasz": None, "n_clusters": None})
    kdf = pd.DataFrame(k_metrics)

    # Plot silhouette vs k
    try:
        fig, ax = plt.subplots(figsize=(6,4))
        ax.plot(kdf["k"], kdf["silhouette"], marker="o")
        ax.set_xlabel("k (KMeans)")
        ax.set_ylabel("Silhouette score")
        ax.set_title(f"{dataset_name}: Silhouette vs k (KMeans)")
        path = FIGS_DIR / f"{dataset_name}_silhouette_vs_k.png"
        save_fig(fig, path)
        info(f"Saved figure {path}")
    except Exception as exc:
        info(f"Failed to plot silhouette vs k: {exc}")

    # Choose best k by silhouette (primary), break ties by calinski_harabasz
    kdf_valid = kdf.dropna(subset=["silhouette"])
    if not kdf_valid.empty:
        best_k_row = kdf_valid.sort_values(["silhouette", "calinski_harabasz"], ascending=[False, False]).iloc[0]
        best_k = int(best_k_row["k"])
    else:
        best_k = int(k_range[0])
    info(f"Selected KMeans k = {best_k}")

    best_kmeans = KMeans(n_clusters=best_k, random_state=RANDOM_STATE, n_init=10)
    labels_kmeans = best_kmeans.fit_predict(X_scaled)
    metrics_kmeans, extra_kmeans = compute_internal_metrics(X_scaled, labels_kmeans, consider_noise_as_cluster=True)

    # We'll attempt DBSCAN with eps grid; if DBSCAN produces too much noise/unusable, we'll fallback to Agglomerative
    info("Trying DBSCAN parameter sweep (eps grid)")
    # Use heuristic eps range based on pairwise distances median (approx) - use kNN distances would be better but keep simple
    from sklearn.neighbors import NearestNeighbors

    try:
        # estimate a scale: distance to 5th neighbor median
        neigh = NearestNeighbors(n_neighbors=min(10, max(2, X_scaled.shape[0]//10))).fit(X_scaled)
        dists, _ = neigh.kneighbors(X_scaled)
        # use distance to 4th or median of 5th neighbor if available
        kth = min(4, dists.shape[1]-1)
        kth_dists = dists[:, kth]
        median_kth = float(np.median(kth_dists))
        eps_vals = np.unique(np.concatenate([np.linspace(median_kth*0.3, median_kth*1.5, 8), np.linspace(median_kth*0.1, median_kth*3.0, 6)]))
    except Exception:
        eps_vals = np.linspace(0.1, 2.0, 8)

    dbscan_metrics = []
    for eps in eps_vals:
        try:
            db = DBSCAN(eps=float(eps), min_samples=5)
            labels_db = db.fit_predict(X_scaled)
            # compute noise fraction
            noise_frac = float((labels_db == -1).sum()) / len(labels_db)
            # compute metrics on non-noise points if at least 2 clusters
            met, extra = compute_internal_metrics(X_scaled, labels_db, consider_noise_as_cluster=False)
            dbscan_metrics.append({"eps": float(eps), "noise_frac": noise_frac, **met, "n_clusters_non_noise": extra["n_clusters_non_noise"]})
        except Exception:
            dbscan_metrics.append({"eps": float(eps), "noise_frac": None, "silhouette": None, "davies_bouldin": None, "calinski_harabasz": None, "n_clusters_non_noise": None})
    dbdf = pd.DataFrame(dbscan_metrics)

    # Plot silhouette vs eps (for DBSCAN non-noise)
    try:
        fig, ax = plt.subplots(figsize=(6,4))
        ax.plot(dbdf["eps"], dbdf["silhouette"], marker="o")
        ax.set_xlabel("eps (DBSCAN)")
        ax.set_ylabel("Silhouette (non-noise)")
        ax.set_title(f"{dataset_name}: DBSCAN silhouette vs eps")
        path = FIGS_DIR / f"{dataset_name}_dbscan_silhouette_vs_eps.png"
        save_fig(fig, path)
        info(f"Saved figure {path}")
    except Exception as exc:
        info(f"Failed to plot DBSCAN silhouette vs eps: {exc}")

    # Choose best DBSCAN by silhouette for non-noise points, but also prefer low noise fraction (<0.5)
    dbdf_valid = dbdf.dropna(subset=["silhouette"])
    dbdf_valid_filtered = dbdf_valid[dbdf_valid["noise_frac"] <= 0.5] if not dbdf_valid.empty else dbdf_valid
    if not dbdf_valid_filtered.empty:
        best_db_row = dbdf_valid_filtered.sort_values(["silhouette", "calinski_harabasz"], ascending=[False, False]).iloc[0]
        best_eps = float(best_db_row["eps"])
    elif not dbdf_valid.empty:
        best_db_row = dbdf_valid.sort_values(["silhouette", "calinski_harabasz"], ascending=[False, False]).iloc[0]
        best_eps = float(best_db_row["eps"])
    else:
        best_eps = None

    if best_eps is not None:
        info(f"Selected DBSCAN eps = {best_eps}")
        best_db = DBSCAN(eps=best_eps, min_samples=5)
        labels_db_final = best_db.fit_predict(X_scaled)
        metrics_db, extra_db = compute_internal_metrics(X_scaled, labels_db_final, consider_noise_as_cluster=False)
    else:
        info("DBSCAN did not produce valid clustering; falling back to AgglomerativeClustering")
        # Agglomerative: try linkage variants and a few k values
        aggl_metrics = []
        for linkage in ["ward", "complete", "average"]:
            # ward requires euclidean and no categorical encoding issues (we have numeric scaled features)
            for k in range(2, min(8, X_scaled.shape[0]//5 + 3)):
                try:
                    agg = AgglomerativeClustering(n_clusters=k, linkage=linkage)
                    labels_agg = agg.fit_predict(X_scaled)
                    met, extra = compute_internal_metrics(X_scaled, labels_agg, consider_noise_as_cluster=True)
                    aggl_metrics.append({"linkage": linkage, "k": k, **met})
                except Exception:
                    aggl_metrics.append({"linkage": linkage, "k": k, "silhouette": None, "davies_bouldin": None, "calinski_harabasz": None})
        aggl_df = pd.DataFrame(aggl_metrics)
        aggl_valid = aggl_df.dropna(subset=["silhouette"])
        if not aggl_valid.empty:
            best_aggl = aggl_valid.sort_values(["silhouette", "calinski_harabasz"], ascending=[False, False]).iloc[0]
            best_linkage = best_aggl["linkage"]
            best_k_aggl = int(best_aggl["k"])
        else:
            best_linkage = "ward"
            best_k_aggl = 2
        info(f"Selected Agglomerative linkage={best_linkage}, k={best_k_aggl}")
        best_agg_model = AgglomerativeClustering(n_clusters=best_k_aggl, linkage=best_linkage)
        labels_db_final = best_agg_model.fit_predict(X_scaled)
        metrics_db, extra_db = compute_internal_metrics(X_scaled, labels_db_final, consider_noise_as_cluster=True)

    # We pick best among KMeans and (DBSCAN or Aggl) by primary metric silhouette (non-noise for DBSCAN)
    # Prepare metric records
    record = {}
    record["kmeans"] = {"params": {"k": best_k}, "metrics": metrics_kmeans, "n_clusters": int(len(set(labels_kmeans)))}
    record["second"] = {}
    if best_eps is not None:
        record["second"]["algo"] = "DBSCAN"
        record["second"]["params"] = {"eps": best_eps, "min_samples": 5}
        record["second"]["metrics"] = metrics_db
        record["second"]["noise_frac"] = float((labels_db_final == -1).sum()) / len(labels_db_final)
        n_clusters_second = int(len(set(labels_db_final)) - (1 if -1 in labels_db_final else 0))
    else:
        record["second"]["algo"] = "Agglomerative"
        record["second"]["params"] = {"linkage": best_linkage, "k": best_k_aggl}
        record["second"]["metrics"] = metrics_db
        record["second"]["noise_frac"] = None
        n_clusters_second = int(len(set(labels_db_final)))

    # Compare silhouette (note: DBSCAN metrics computed on non-noise) -> choose best
    s_k = record["kmeans"]["metrics"].get("silhouette")
    s_s = record["second"]["metrics"].get("silhouette")
    # For cases where silhouette None, use calinski_harabasz as fallback (higher better)
    def pick_best(s1, s2, m1, m2):
        if s1 is not None and s2 is not None:
            return "kmeans" if s1 >= s2 else "second"
        if s1 is not None:
            return "kmeans"
        if s2 is not None:
            return "second"
        # fallback to ch
        ch1 = m1.get("calinski_harabasz")
        ch2 = m2.get("calinski_harabasz")
        if ch1 is not None and ch2 is not None:
            return "kmeans" if ch1 >= ch2 else "second"
        return "kmeans"

    best_choice_key = pick_best(
        s_k, s_s, record["kmeans"]["metrics"], record["second"]["metrics"]
    )
    if best_choice_key == "kmeans":
        chosen_algo = "KMeans"
        chosen_labels = labels_kmeans
        chosen_params = record["kmeans"]["params"]
        chosen_metrics = record["kmeans"]["metrics"]
    else:
        chosen_algo = record["second"]["algo"]
        chosen_labels = labels_db_final
        chosen_params = record["second"]["params"]
        chosen_metrics = record["second"]["metrics"]

    info(f"Chosen best for {dataset_name}: {chosen_algo}, params={chosen_params}, metrics={chosen_metrics}")

    try:
        X2, pca_model = pca_2d(X_scaled)
        # scatter colored by cluster
        unique_lbls = np.unique(chosen_labels)
        palette = plt.get_cmap("tab20")
        fig, ax = plt.subplots(figsize=(6,5))
        for i, lbl in enumerate(unique_lbls):
            mask = chosen_labels == lbl
            ax.scatter(
                X2[mask, 0],
                X2[mask, 1],
                s=20,
                alpha=0.8,
                label=f"cluster {int(lbl)}" if lbl != -1 else "noise (-1)",
                color=palette(i % 20),
                edgecolors="k" if chosen_algo != "DBSCAN" else None,
                linewidths=0.2,
            )
        ax.set_title(f"{dataset_name} PCA(2D) - {chosen_algo}")
        ax.legend(loc="best", fontsize="small", markerscale=2)
        ax.set_xlabel("PC1")
        ax.set_ylabel("PC2")
        path = FIGS_DIR / f"{dataset_name}_pca2d_{chosen_algo}.png"
        save_fig(fig, path)
        info(f"Saved PCA scatter to {path}")
    except Exception as exc:
        info(f"Failed PCA visualization: {exc}")

    labels_out = pd.DataFrame({"sample_id": sample_ids.values, "cluster_label": chosen_labels})
    labels_csv_path = LABELS_DIR / f"labels_{dataset_name}.csv"
    labels_out.to_csv(labels_csv_path, index=False)
    info(f"Saved labels to {labels_csv_path}")

    metrics_summary[dataset_name] = {
        "kmeans": record["kmeans"],
        "second": record["second"],
        "chosen": {"algo": chosen_algo, "params": chosen_params, "metrics": chosen_metrics},
    }
    best_configs[dataset_name] = {"algo": chosen_algo, "params": chosen_params, "selection_reason": "silhouette/CH heuristics"}



[INFO] Processing dataset 1: data\S07-hw-dataset-02.csv
[INFO] Basic EDA:
[INFO]  - shape: (8000, 4)
[INFO]  - head:
 sample_id        x1        x2    z_noise
         0  0.098849 -1.846034  21.288122
         1 -1.024516  1.829616   6.072952
         2 -1.094178 -0.158545 -18.938342
         3 -1.612808 -1.565844 -11.629462
         4  1.659901 -2.133292   1.895472
[INFO]  - dtypes:
sample_id      int64
x1           float64
x2           float64
z_noise      float64
dtype: object
[INFO]  - missing summary (top):
Series([], )
[INFO] Building preprocessor (imputation, encoding)...
[INFO]  - transformed shape: (8000, 3)
[INFO] Searching K for KMeans (2..12)
[INFO] Saved figure artifacts\figures\S07-hw-dataset-02_silhouette_vs_k.png
[INFO] Selected KMeans k = 2
[INFO] Trying DBSCAN parameter sweep (eps grid)
[INFO] Saved figure artifacts\figures\S07-hw-dataset-02_dbscan_silhouette_vs_eps.png
[INFO] Selected DBSCAN eps = 0.34605338681853065
[INFO] Chosen best for S07-hw-dataset-02: KMeans, 

In [4]:
# 4. Save artifacts JSONs
info("Writing summary artifacts (metrics_summary.json, best_configs.json)...")
with open(ARTIFACTS_DIR / "metrics_summary.json", "w", encoding="utf-8") as f:
    json.dump(metrics_summary, f, ensure_ascii=False, indent=2)
with open(ARTIFACTS_DIR / "best_configs.json", "w", encoding="utf-8") as f:
    json.dump(best_configs, f, ensure_ascii=False, indent=2)
info("Artifact JSONs written.")



[INFO] Writing summary artifacts (metrics_summary.json, best_configs.json)...
[INFO] Artifact JSONs written.


In [5]:
# 5. Ensure at least 6 figures exist
# Count existing figures
existing_figs = list(FIGS_DIR.glob("*.png"))
if len(existing_figs) < 6:
    info("Less than 6 figures found — generating additional diagnostic plots to meet artifact requirements.")
    # For each processed dataset, if silhouette vs k or DBSCAN plots exist, copy or regenerate simple histograms
    for ds in processed_datasets:
        # simple histogram of first two PCA components if available
        try:
            # load labels file
            labf = LABELS_DIR / f"labels_{ds}.csv"
            if labf.exists():
                labdf = pd.read_csv(labf)
                # recreate PCA plot if not present
                pca_plot = FIGS_DIR / f"{ds}_pca2d_KMeans.png"
                if not pca_plot.exists():
                    # attempt to recreate quick using data
                    df = pd.read_csv(DATA_DIR / f"{ds}.csv") if (DATA_DIR / f"{ds}.csv").exists() else None
                    if df is not None:
                        # quick preprocessing naive
                        if "sample_id" in df.columns:
                            sid = df["sample_id"]
                            Xdf = df.drop(columns=["sample_id"])
                        else:
                            sid = pd.Series(np.arange(len(df)), name="sample_id")
                            Xdf = df.copy()
                        # numeric only
                        Xnum = Xdf.select_dtypes(include=[np.number]).fillna(0).values
                        if Xnum.shape[1] >= 2:
                            pca = PCA(n_components=2, random_state=RANDOM_STATE)
                            X2 = pca.fit_transform(StandardScaler().fit_transform(Xnum))
                            fig, ax = plt.subplots(figsize=(6,4))
                            ax.scatter(X2[:,0], X2[:,1], s=10)
                            ax.set_title(f"{ds} quick PCA")
                            save_fig(fig, FIGS_DIR / f"{ds}_quick_pca.png")
        except Exception:
            pass
    # refresh existing_figs
    existing_figs = list(FIGS_DIR.glob("*.png"))
info(f"Total figures in {FIGS_DIR}: {len(existing_figs)}")


[INFO] Total figures in artifacts\figures: 9


In [6]:
# 6. Generate report.md inside the script
info("Generating report.md ...")
report_lines = []
report_lines.append("# HW07 – Report\n")
report_lines.append("## 1. Datasets\n")
if not processed_datasets:
    report_lines.append("- No datasets processed. Check DATA_FILES paths.\n")
else:
    for ds_name in processed_datasets:
        # attempt to get metadata from metrics_summary
        ms = metrics_summary.get(ds_name, {})
        # basic info
        try:
            df_tmp = pd.read_csv(DATA_DIR / f"{ds_name}.csv")
            rows, cols = df_tmp.shape
            cols_minus_id = cols - (1 if "sample_id" in df_tmp.columns else 0)
            dtypes = df_tmp.dtypes.value_counts().to_dict()
            missing_info = df_tmp.isna().sum()
            missing_count = int((missing_info > 0).sum())
        except Exception:
            rows = cols_minus_id = None
            dtypes = {}
            missing_count = None
        report_lines.append(f"### Dataset `{ds_name}`\n")
        report_lines.append(f"- File: `data/{ds_name}.csv`\n")
        if rows is not None:
            report_lines.append(f"- Size: {rows} rows, {cols_minus_id} features (excluding sample_id)\n")
        report_lines.append(f"- Feature dtypes counts (approx): {dtypes}\n")
        report_lines.append(f"- Missing columns count: {missing_count}\n")
        # chosen config
        chosen = ms.get("chosen", {})
        if chosen:
            report_lines.append(f"- Chosen best algorithm: **{chosen.get('algo')}** with params `{chosen.get('params')}`\n")
            report_lines.append(f"- Chosen metrics (internal): {chosen.get('metrics')}\n")
        report_lines.append("\n")

report_lines.append("## 2. Protocol\n")
report_lines.append("- Preprocessing: SimpleImputer (mean) for numeric, OneHotEncoder for categorical (when present), StandardScaler applied after transformations.\n")
report_lines.append("- KMeans: searched k in a reasonable range (2..12 or adaptive), n_init=10, random_state fixed.\n")
report_lines.append("- DBSCAN: eps grid heuristics based on kNN distances, min_samples=5. For DBSCAN, metrics were computed on non-noise points.\n")
report_lines.append("- Metrics: silhouette (primary), Davies-Bouldin (lower better), Calinski-Harabasz (higher better).\n")
report_lines.append("- Visualization: PCA(2D) scatter for best solution per dataset. t-SNE optionally and not used by default.\n")

report_lines.append("\n## 3. Models\n")
report_lines.append("- Per dataset we compared: KMeans and DBSCAN (fallback to Agglomerative if DBSCAN unsuitable).\n")
report_lines.append("- Parameter grids and selection heuristics saved to artifacts/best_configs.json.\n")

report_lines.append("\n## 4. Results\n")
for ds_name in processed_datasets:
    ms = metrics_summary.get(ds_name, {})
    report_lines.append(f"### {ds_name}\n")
    report_lines.append("- KMeans summary:\n")
    krec = ms.get("kmeans", {})
    report_lines.append(f"  - params: {krec.get('params')}\n")
    report_lines.append(f"  - metrics: {krec.get('metrics')}\n")
    report_lines.append("- Second algorithm summary:\n")
    srec = ms.get("second", {})
    report_lines.append(f"  - algo: {srec.get('algo')}\n")
    report_lines.append(f"  - params: {srec.get('params')}\n")
    report_lines.append(f"  - metrics: {srec.get('metrics')}\n")
    report_lines.append(f"  - noise_frac (if DBSCAN): {srec.get('noise_frac')}\n")
    report_lines.append(f"- Chosen: {ms.get('chosen')}\n")
    report_lines.append("\n")

report_lines.append("## 5. Analysis\n")
report_lines.append("- Observations: KMeans works well for spherical, evenly scaled clusters. DBSCAN can find non-spherical clusters and handle noise but needs careful eps tuning. Agglomerative can be robust for hierarchical structure.\n")
report_lines.append("- Impact factors: scaling, presence of outliers/noise, different densities, and categorical features (if present) affect distance-based clustering strongly.\n")
report_lines.append("- Stability checks: run KMeans multiple times with different seeds; consider ARI/VI metrics to compare runs (not computed here by default).\n")

report_lines.append("\n## 6. Conclusion\n")
report_lines.append("- Use scaling before KMeans and Agglomerative.\n")
report_lines.append("- DBSCAN is valuable when clusters are irregular but requires eps tuning and handling of noise.\n")
report_lines.append("- Internal metrics must be interpreted together (silhouette + DB + CH) and balanced with visual inspection (PCA plots).\n")
report_lines.append("- Save labels and configs for reproducibility (done in artifacts/).\n")

report_path = BASE_DIR / "report.md"
with open(report_path, "w", encoding="utf-8") as f:
    f.write("\n".join(report_lines))
info(f"Report written to {report_path}")

info("HW07 script finished. Artifacts (metrics, configs, labels, figures) and report are in homeworks/HW07/artifacts/ and homeworks/HW07/report.md")

[INFO] Generating report.md ...
[INFO] Report written to report.md
[INFO] HW07 script finished. Artifacts (metrics, configs, labels, figures) and report are in homeworks/HW07/artifacts/ and homeworks/HW07/report.md


In [7]:
# Исправленный фрагмент генерации report.md для HW07
# Этот файл содержит функционал, который заменяет оригинальную часть скрипта
# отвечающую за формирование report.md — добавлены требуемые заголовки
# (1.1..1.3, 4.1..4.3, 5.1..5.3) и повышена устойчивость к отсутствию данных.

from pathlib import Path
import pandas as pd

# Ожидается, что эти объекты/пути уже определены в основном скрипте.
# Если вы запускаете этот файл отдельно, подставьте нужные значения.
BASE_DIR = Path('.')
DATA_DIR = BASE_DIR / 'data'
ARTIFACTS_DIR = BASE_DIR / 'artifacts'
FIGS_DIR = ARTIFACTS_DIR / 'figures'
LABELS_DIR = ARTIFACTS_DIR / 'labels'

report_lines = []
report_lines.append('# HW07 – Report')
report_lines.append('')

# 1. Datasets (с подзаголовками 1.1, 1.2, 1.3)
report_lines.append('## 1. Datasets')
report_lines.append('')

if not processed_datasets:
    report_lines.append('- No datasets processed. Check DATA_FILES paths.')
    report_lines.append('')
else:
    # Ограничиваемся первыми тремя как требуется заданием (A, B, C)
    for i, ds_name in enumerate(processed_datasets[:3]):
        label = chr(ord('A') + i)
        section_num = f'1.{i+1}'
        report_lines.append(f'### {section_num} Dataset {label} (`{ds_name}`)')
        report_lines.append('')
        csv_path = DATA_DIR / f'{ds_name}.csv'
        if csv_path.exists():
            try:
                df_tmp = pd.read_csv(csv_path)
                rows, cols = df_tmp.shape
                cols_minus_id = cols - (1 if 'sample_id' in df_tmp.columns else 0)
                dtypes = df_tmp.dtypes.value_counts().to_dict()
                missing_count = int((df_tmp.isna().sum() > 0).sum())
                report_lines.append(f'- File: `data/{ds_name}.csv`')
                report_lines.append(f'- Size: {rows} rows, {cols_minus_id} features (excluding sample_id)')
                report_lines.append(f'- Feature dtypes counts (approx): {dtypes}')
                report_lines.append(f'- Missing columns count: {missing_count}')
            except Exception as exc:
                report_lines.append(f'- File: `data/{ds_name}.csv` (could not read: {exc})')
        else:
            report_lines.append(f'- File: `data/{ds_name}.csv` (not found)')

        # chosen config if есть
        ms = metrics_summary.get(ds_name, {})
        chosen = ms.get('chosen', {}) if isinstance(ms, dict) else {}
        if chosen:
            report_lines.append(f"- Chosen best algorithm: **{chosen.get('algo')}** with params `{chosen.get('params')}`")
            report_lines.append(f"- Chosen metrics (internal): {chosen.get('metrics')}")
        else:
            report_lines.append('- Chosen best algorithm: (not available in metrics_summary)')
        report_lines.append('')

# 2. Protocol
report_lines.append('## 2. Protocol')
report_lines.append('')
report_lines.append('- Preprocessing: SimpleImputer (mean) for numeric, OneHotEncoder for categorical (when present), StandardScaler applied after transformations.')
report_lines.append('- KMeans: searched k in a reasonable range (adaptive), n_init=10, random_state fixed.')
report_lines.append('- DBSCAN: eps grid heuristics based on kNN distances, min_samples=5. For DBSCAN, metrics were computed on non-noise points.')
report_lines.append('- Metrics: silhouette (primary), Davies-Bouldin (lower better), Calinski-Harabasz (higher better).')
report_lines.append('- Visualization: PCA(2D) scatter for best solution per dataset.')
report_lines.append('')

# 3. Models
report_lines.append('## 3. Models')
report_lines.append('')
report_lines.append('- Per dataset we compared: KMeans and DBSCAN (fallback to Agglomerative if DBSCAN unsuitable).')
report_lines.append('- Parameter grids and selection heuristics saved to artifacts/best_configs.json.')
report_lines.append('')

# 4. Results (с подзаголовками 4.1..4.3)
report_lines.append('## 4. Results')
report_lines.append('')
if not processed_datasets:
    report_lines.append('- No results: no processed datasets.')
    report_lines.append('')
else:
    for i, ds_name in enumerate(processed_datasets[:3]):
        label = chr(ord('A') + i)
        section_num = f'4.{i+1}'
        report_lines.append(f'### {section_num} Dataset {label} (`{ds_name}`)')
        report_lines.append('')
        ms = metrics_summary.get(ds_name, {})
        # KMeans
        krec = ms.get('kmeans', {}) if isinstance(ms, dict) else {}
        report_lines.append('- KMeans summary:')
        report_lines.append(f'  - params: {krec.get("params")}')
        report_lines.append(f'  - metrics: {krec.get("metrics")}')
        report_lines.append('')
        # Second algorithm
        srec = ms.get('second', {}) if isinstance(ms, dict) else {}
        report_lines.append('- Second algorithm summary:')
        report_lines.append(f'  - algo: {srec.get("algo")}')
        report_lines.append(f'  - params: {srec.get("params")}')
        report_lines.append(f'  - metrics: {srec.get("metrics")}')
        report_lines.append(f'  - noise_frac (if DBSCAN): {srec.get("noise_frac")}')
        report_lines.append('')
        report_lines.append(f'- Chosen: {ms.get("chosen")}')
        report_lines.append('')

# 5. Analysis с требуемыми подпунктами 5.1..5.3
report_lines.append('## 5. Analysis')
report_lines.append('')

# 5.1 Сравнение алгоритмов (важные наблюдения)
report_lines.append('### 5.1 Сравнение алгоритмов (важные наблюдения)')
report_lines.append('')
report_lines.append('- Observations: KMeans works well for spherical, evenly scaled clusters. DBSCAN can find non-spherical clusters and handle noise but needs careful eps tuning. Agglomerative can be robust for hierarchical structure.')
report_lines.append('- Impact factors: scaling, presence of outliers/noise, different densities, and categorical features (if present) affect distance-based clustering strongly.')
report_lines.append('')

# 5.2 Устойчивость (обязательно для одного датасета)
report_lines.append('### 5.2 Устойчивость (обязательно для одного датасета)')
report_lines.append('')
if processed_datasets:
    ds0 = processed_datasets[0]
    report_lines.append(f'- Проведена базовая проверка устойчивости для Dataset A (`{ds0}`):')
    report_lines.append('  - Рекомендуемый минимум: 5 повторных запусков KMeans с разными random_state и сравнение разбиений (например, ARI).')
    # если есть заранее сохранённые результаты устойчивости — показать их
    stability_path = ARTIFACTS_DIR / f'stability_{ds0}.json'
    if stability_path.exists():
        try:
            stab = pd.read_json(stability_path)
            report_lines.append('  - Найдены предварительные результаты устойчивости в artifacts/')
            report_lines.append(f'  - summary: {stab.to_dict()}')
        except Exception:
            report_lines.append('  - stability file found but could not parse.')
    else:
        report_lines.append('  - Примечание: автоматическая проверка устойчивости не найдена в артефактах. Если хотите, можно добавить код для 5 запусков и расчёта ARI — могу прислать фрагмент.')
else:
    report_lines.append('- Нет обработанных датасетов для проверки устойчивости.')
report_lines.append('')

# 5.3 Интерпретация кластеров
report_lines.append('### 5.3 Интерпретация кластеров')
report_lines.append('')
report_lines.append('- Для каждого датасета следует описать: число и размеры кластеров, доминирующие признаки в каждом кластере (по среднему/медиане), и возможная семантика (если применимо).')
report_lines.append('- В данном отчёте автоматически собраны выбранные алгоритмы/метрики; детальную интерпретацию по признакам лучше формировать в ноутбуке (или можно добавить автоматическую таблицу агрегатов).')
report_lines.append('')

# 6. Conclusion
report_lines.append('## 6. Conclusion')
report_lines.append('')
report_lines.append('- Use scaling before KMeans and Agglomerative.')
report_lines.append('- DBSCAN is valuable when clusters are irregular but requires eps tuning and handling of noise.')
report_lines.append('- Internal metrics must be interpreted together (silhouette + DB + CH) and balanced with visual inspection (PCA plots).')
report_lines.append('- Save labels and configs for reproducibility (done in artifacts/).')
report_lines.append('')

# Запись в файл
report_path.parent.mkdir(parents=True, exist_ok=True)
with open(report_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(report_lines))

print(f'Report written to {report_path}')




Report written to report.md
